In [1]:
#!/usr/bin/python
# -*- encoding: utf-8 -*-
from logger import setup_logger
from models.model_stages import BiSeNet
from cityscapes import CityScapes
from loss.loss import OhemCELoss
from loss.detail_loss import DetailAggregateLoss
from evaluation import MscEvalV0
from optimizer_loss import Optimizer

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.distributed as dist

In [3]:
import os
import os.path as osp
import logging
import time
import datetime
import argparse

In [4]:
logger = logging.getLogger()

In [5]:
def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Unsupported value encountered.')

In [6]:
def parse_args():
    parse = argparse.ArgumentParser()
    parse.add_argument(
            '--local_rank',
            dest = 'local_rank',
            type = int,
            default = -1,
            )
    parse.add_argument(
            '--n_workers_train',
            dest = 'n_workers_train',
            type = int,
            default = 12,
            )
    parse.add_argument(
            '--n_workers_val',
            dest = 'n_workers_val',
            type = int,
            default = 1,
            )
    parse.add_argument(
            '--n_img_per_gpu',
            dest = 'n_img_per_gpu',
            type = int,
            default = 16,
            )
    parse.add_argument(
            '--max_iter',
            dest = 'max_iter',
            type = int,
            default = 60000,
            )
    parse.add_argument(
            '--save_iter_sep',
            dest = 'save_iter_sep',
            type = int,
            default = 1000,
            )
    parse.add_argument(
            '--warmup_steps',
            dest = 'warmup_steps',
            type = int,
            default = 1000,
            )      
    parse.add_argument(
            '--mode',
            dest = 'mode',
            type = str,
            default = 'train',
            )
    parse.add_argument(
            '--ckpt',
            dest = 'ckpt',
            type = str,
            default = None,
            )
    parse.add_argument(
            '--respath',
            dest = 'respath',
            type = str,
            default = '/xiaoou/STDC-Seg-master/STDC-Seg-master/checkpoint/train_STDC_seg/',
            )
    parse.add_argument(
            '--backbone',
            dest = 'backbone',
            type = str,
            default = 'STDCNet1446',
            )
    parse.add_argument(
            '--pretrain_path',
            dest = 'pretrain_path',
            type = str,
            default = '/xiaoou/STDC-Seg-master/STDC-Seg-master/STDC-Seg-weight/STDCNet1446_76.47.tar',
            )
    parse.add_argument(
            '--use_conv_last',
            dest = 'use_conv_last',
            type = str2bool,
            default = False,
            )
    parse.add_argument(
            '--use_boundary_2',
            dest = 'use_boundary_2',
            type = str2bool,
            default = False,
            )
    parse.add_argument(
            '--use_boundary_4',
            dest = 'use_boundary_4',
            type = str2bool,
            default = False,
            )
    parse.add_argument(
            '--use_boundary_8',
            dest = 'use_boundary_8',
            type = str2bool,
            default = True,
            )
    parse.add_argument(
            '--use_boundary_16',
            dest = 'use_boundary_16',
            type = str2bool,
            default = False,
            )
    args = parse.parse_args(args=[])
    return args

In [7]:
import sys

In [8]:
sys.path.append('..')

In [9]:
import get_camvid_loader

In [10]:
from configs import set_cfg_from_file

In [11]:
parse = argparse.ArgumentParser()
parse.add_argument('--config', dest='config', type=str,
        default='/xiaoou/STDC-Seg-master/configs/need.py',)
parse.add_argument('--finetune-from', type=str, default=None,)

args_0 = parse.parse_known_args()[0]

In [12]:
cfg = set_cfg_from_file(args_0.config)

In [13]:
args = parse_args()

In [14]:
args

Namespace(backbone='STDCNet1446', ckpt=None, local_rank=-1, max_iter=60000, mode='train', n_img_per_gpu=16, n_workers_train=12, n_workers_val=1, pretrain_path='/xiaoou/STDC-Seg-master/STDC-Seg-master/STDC-Seg-weight/STDCNet1446_76.47.tar', respath='/xiaoou/STDC-Seg-master/STDC-Seg-master/checkpoint/train_STDC_seg/', save_iter_sep=1000, use_boundary_16=False, use_boundary_2=False, use_boundary_4=False, use_boundary_8=True, use_conv_last=False, warmup_steps=1000)

In [15]:
print(bbb)

NameError: name 'bbb' is not defined

In [16]:
n_classes = 11
n_img_per_gpu = args.n_img_per_gpu
n_workers_train = args.n_workers_train
n_workers_val = args.n_workers_val
use_boundary_16 = args.use_boundary_16
use_boundary_8 = args.use_boundary_8
use_boundary_4 = args.use_boundary_4
use_boundary_2 = args.use_boundary_2

mode = args.mode
cropsize = [720,960]
randomscale = (0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 1.125, 1.25, 1.375, 1.5)


In [17]:
ignore_idx = 255
net = BiSeNet(backbone=args.backbone, n_classes=n_classes, pretrain_model=args.pretrain_path, 
use_boundary_2=use_boundary_2, use_boundary_4=use_boundary_4, use_boundary_8=use_boundary_8, 
use_boundary_16=use_boundary_16, use_conv_last=args.use_conv_last)

/xiaoou/STDC-Seg-master/STDC-Seg-master/models/model_stages.py:129: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.W.weight, 0)
/xiaoou/STDC-Seg-master/STDC-Seg-master/models/model_stages.py:130: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.W.bias, 0)


In [18]:
state = torch.load('/xiaoou/STDC-Seg-master/STDC-Seg-master/checkpoint/train_STDC_seg/pths/model_final.pth')

In [19]:
net.load_state_dict(state)

<All keys matched successfully>

In [ ]:
import time

In [36]:
dlval = get_camvid_loader.get_data_loader(cfg, mode='test')

In [ ]:
times = []

In [ ]:
for i,data in enumerate(dlval):
    img,label = data
    if i == 5:
        break

In [ ]:
outs = net(img.cuda())

In [ ]:
for i in range(len(outs)):
    print(outs[i].shape)

In [ ]:
outs = outs[0][1]

In [ ]:
outs  =outs.squeeze(0)

In [ ]:
outs = outs.cpu().detach().numpy()

In [ ]:
outs = outs.max(0)[1]

In [ ]:
outs.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(outs)

In [ ]:
label = label.squeeze(0)

In [26]:
from torch.autograd import Variable

In [27]:
import os, sys
import cv2
import numpy as np

from multiprocessing import Pool
# import copy_reg
import copyreg
import types


def _pickle_method(m):
    if m.im_self is None:
        return getattr, (m.im_class, m.im_func.func_name)
    else:
        return getattr, (m.im_self, m.im_func.func_name)


copyreg.pickle(types.MethodType, _pickle_method)


class ConfusionMatrix(object):

    def __init__(self, nclass, classes=None, ignore_label=255):
        self.nclass = nclass
        self.classes = classes
        self.M = np.zeros((nclass, nclass))
        self.ignore_label = ignore_label

    def add(self, gt, pred):
        assert (np.max(pred) <= self.nclass)
        assert (len(gt) == len(pred))
        for i in range(len(gt)):
            if not gt[i] == self.ignore_label:
                self.M[gt[i], pred[i]] += 1.0

    def addM(self, matrix):
        assert (matrix.shape == self.M.shape)
        self.M += matrix

    def __str__(self):
        pass

    # Pii为预测正确的数量，Pij和Pji分别被解释为假正和假负，尽管两者都是假正与假负之和
    def recall(self):  # 预测为正确的像素中确认为正确像素的个数
        recall = 0.0
        for i in range(self.nclass):
            recall += self.M[i, i] / np.sum(self.M[:, i])

        return recall / self.nclass

    def accuracy(self):  # 分割正确的像素除以总像素
        accuracy = 0.0
        for i in range(self.nclass):
            accuracy += self.M[i, i] / np.sum(self.M[i, :])

        return accuracy / self.nclass

    # 雅卡尔指数，又称为交并比（IOU）
    def jaccard(self):
        jaccard = 0.0
        jaccard_perclass = []
        for i in range(self.nclass):
            if not self.M[i, i] == 0:
                jaccard_perclass.append(self.M[i, i] / (np.sum(self.M[i, :]) + np.sum(self.M[:, i]) - self.M[i, i]))

        return np.sum(jaccard_perclass) / len(jaccard_perclass), jaccard_perclass, self.M

    def generateM(self, item):
        gt, pred = item
        m = np.zeros((self.nclass, self.nclass))
        assert (len(gt) == len(pred))
        for i in range(len(gt)):
            if gt[i] < self.nclass:  # and pred[i] < self.nclass:
                m[gt[i], pred[i]] += 1.0
        return m


def get_iou(data_list, class_num, save_path=None):
    """ 
    Args:
      data_list: a list, its elements [gt, output]
      class_num: the number of label
    """
    from multiprocessing import Pool

    ConfM = ConfusionMatrix(class_num)
    f = ConfM.generateM
    pool = Pool()
    m_list = pool.map(f, data_list)
    pool.close()
    pool.join()

    for m in m_list:
        ConfM.addM(m)

    aveJ, j_list, M = ConfM.jaccard()
    # print(j_list)
    # print(M)
    # print('meanIOU: ' + str(aveJ) + '\n')

    if save_path:
        with open(save_path, 'w') as f:
            f.write('meanIOU: ' + str(aveJ) + '\n')
            f.write(str(j_list) + '\n')
            f.write(str(M) + '\n')
    return aveJ, j_list


In [32]:
def val(val_loader, model):
    """
    args:
      val_loader: loaded for validation dataset
      model: model
    return: mean IoU and IoU class
    """
    # evaluation mode
    model.eval()
    total_batches = len(val_loader)

    data_list = []
    for i, (input, label, size, name) in enumerate(val_loader):
        with torch.no_grad():
            input_var = Variable(input).cuda()
        start_time = time.time()
        output = model(input_var)
        time_taken = time.time() - start_time
        print("[%d/%d]  time: %.2f" % (i + 1, total_batches, time_taken))
        output = output[0].cpu().data[0].numpy()
        gt = np.asarray(label[0].numpy(), dtype=np.uint8)
        output = output.transpose(1, 2, 0)
        output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)
        data_list.append([gt.flatten(), output.flatten()])

    meanIoU, per_class_iu = get_iou(data_list,11)
    return meanIoU, per_class_iu

In [30]:
net.cuda()

BiSeNet(
  (cp): ContextPath(
    (backbone): STDCNet1446(
      (features): Sequential(
        (0): ConvX(
          (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
        )
        (1): ConvX(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
        )
        (2): CatBottleneck(
          (conv_list): ModuleList(
            (0): ConvX(
              (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (relu): ReLU(inplace=True)
            )
            (1): ConvX(
              (conv): Conv2d(128, 64, kern

In [37]:
a,b = val(dlval,net)

[1/117]  time: 0.04
[2/117]  time: 0.02
[3/117]  time: 0.03
[4/117]  time: 0.02
[5/117]  time: 0.02
[6/117]  time: 0.02
[7/117]  time: 0.02
[8/117]  time: 0.02
[9/117]  time: 0.02
[10/117]  time: 0.02
[11/117]  time: 0.02
[12/117]  time: 0.04
[13/117]  time: 0.02
[14/117]  time: 0.02
[15/117]  time: 0.02
[16/117]  time: 0.02
[17/117]  time: 0.02
[18/117]  time: 0.02
[19/117]  time: 0.03
[20/117]  time: 0.02
[21/117]  time: 0.02
[22/117]  time: 0.02
[23/117]  time: 0.02
[24/117]  time: 0.02
[25/117]  time: 0.02
[26/117]  time: 0.02
[27/117]  time: 0.03
[28/117]  time: 0.02
[29/117]  time: 0.02
[30/117]  time: 0.02
[31/117]  time: 0.02
[32/117]  time: 0.02
[33/117]  time: 0.02
[34/117]  time: 0.02
[35/117]  time: 0.03
[36/117]  time: 0.02
[37/117]  time: 0.02
[38/117]  time: 0.02
[39/117]  time: 0.02
[40/117]  time: 0.03
[41/117]  time: 0.02
[42/117]  time: 0.02
[43/117]  time: 0.03
[44/117]  time: 0.02
[45/117]  time: 0.02
[46/117]  time: 0.02
[47/117]  time: 0.02
[48/117]  time: 0.03
[

In [38]:
a

0.6962630819837663

In [39]:
b

[0.9264271022427017,
 0.8369398213519287,
 0.35921857109530786,
 0.953257006725939,
 0.8335900912585187,
 0.7535265453413517,
 0.47566823911204503,
 0.4167501566903252,
 0.8413053994350178,
 0.6236582539341279,
 0.6385527146341653]

In [ ]:
def train():
    args = parse_args()
    print(args)
    save_pth_path = os.path.join(args.respath, 'pths')
    dspth = '/xiaoou/my_last_project/dataset/cityscapes'
    
    # print(save_pth_path)
    # print(osp.exists(save_pth_path))
    # if not osp.exists(save_pth_path) and dist.get_rank()==0: 
    if not osp.exists(save_pth_path):
        os.makedirs(save_pth_path)
    
    
#     torch.cuda.set_device(args.local_rank)
#     dist.init_process_group(
#                 backend = 'nccl',
#                 init_method = 'tcp://127.0.0.1:33274',
#                 world_size = torch.cuda.device_count(),
#                 rank=args.local_rank
#                 )
    
    setup_logger(args.respath)
    ## dataset
    n_classes = 11
    n_img_per_gpu = args.n_img_per_gpu
    n_workers_train = args.n_workers_train
    n_workers_val = args.n_workers_val
    use_boundary_16 = args.use_boundary_16
    use_boundary_8 = args.use_boundary_8
    use_boundary_4 = args.use_boundary_4
    use_boundary_2 = args.use_boundary_2
    
    mode = args.mode
    cropsize = [720,960]
    randomscale = (0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 1.125, 1.25, 1.375, 1.5)

#     if dist.get_rank()==0: 
#         logger.info('n_workers_train: {}'.format(n_workers_train))
#         logger.info('n_workers_val: {}'.format(n_workers_val))
#         logger.info('use_boundary_2: {}'.format(use_boundary_2))
#         logger.info('use_boundary_4: {}'.format(use_boundary_4))
#         logger.info('use_boundary_8: {}'.format(use_boundary_8))
#         logger.info('use_boundary_16: {}'.format(use_boundary_16))
#         logger.info('mode: {}'.format(args.mode))
    
    
#     ds = CityScapes(dspth, cropsize=cropsize, mode=mode, randomscale=randomscale)
# #     sampler = torch.utils.data.distributed.DistributedSampler(ds)
#     dl = DataLoader(ds,
#                     batch_size = n_img_per_gpu,
#                     shuffle = False,
# #                     sampler = ds,
#                     num_workers = n_workers_train,
#                     pin_memory = False,
#                     drop_last = True)
#     # exit(0)
#     dsval = CityScapes(dspth, mode='val', randomscale=randomscale)
# #     sampler_val = torch.utils.data.distributed.DistributedSampler(dsval)
#     dlval = DataLoader(dsval,
#                     batch_size = 2,
#                     shuffle = False,
# #                     sampler = sampler_val,
#                     num_workers = n_workers_val,
#                     drop_last = False)

    ## model
    dl = get_camvid_loader.get_data_loader(cfg, mode='train')

    dlval = get_camvid_loader.get_data_loader(cfg, mode='val')
    
    
    
    ignore_idx = 255
    net = BiSeNet(backbone=args.backbone, n_classes=n_classes, pretrain_model=args.pretrain_path, 
    use_boundary_2=use_boundary_2, use_boundary_4=use_boundary_4, use_boundary_8=use_boundary_8, 
    use_boundary_16=use_boundary_16, use_conv_last=args.use_conv_last)

    if not args.ckpt is None:
        net.load_state_dict(torch.load(args.ckpt, map_location='cpu'))
    net.cuda()
    net.train()
#     net = nn.parallel.DistributedDataParallel(net,
#             device_ids = [args.local_rank, ],
#             output_device = args.local_rank,
#             find_unused_parameters=True
#             )

    score_thres = 0.7
    n_min = n_img_per_gpu*cropsize[0]*cropsize[1]//16
    criteria_p = OhemCELoss(thresh=score_thres, n_min=n_min, ignore_lb=ignore_idx)
    criteria_16 = OhemCELoss(thresh=score_thres, n_min=n_min, ignore_lb=ignore_idx)
    criteria_32 = OhemCELoss(thresh=score_thres, n_min=n_min, ignore_lb=ignore_idx)
    boundary_loss_func = DetailAggregateLoss()
    ## optimizer
    maxmIOU50 = 0.
    maxmIOU75 = 0.
    momentum = 0.9
    weight_decay = 5e-4
    lr_start = 1e-2
    max_iter = args.max_iter
    save_iter_sep = args.save_iter_sep
    power = 0.9
    warmup_steps = args.warmup_steps
    warmup_start_lr = 1e-5

#     if dist.get_rank()==0: 
#         print('max_iter: ', max_iter)
#         print('save_iter_sep: ', save_iter_sep)
#         print('warmup_steps: ', warmup_steps)
    optim = Optimizer(
#             model = net.module,
            model = net,
            loss = boundary_loss_func,
            lr0 = lr_start,
            momentum = momentum,
            wd = weight_decay,
            warmup_steps = warmup_steps,
            warmup_start_lr = warmup_start_lr,
            max_iter = max_iter,
            power = power)
    
    ## train loop
    msg_iter = 50
    loss_avg = []
    loss_boundery_bce = []
    loss_boundery_dice = []
    st = glob_st = time.time()
    diter = iter(dl)
    epoch = 0
    for it in range(max_iter):
        try:
            im, lb = next(diter)
            if not im.size()[0]==n_img_per_gpu: raise StopIteration
        except StopIteration:
            epoch += 1
#             sampler.set_epoch(epoch)
            diter = iter(dl)
            im, lb = next(diter)
        im = im.cuda()
        lb = lb.cuda()
        H, W = im.size()[2:]
        lb = torch.squeeze(lb, 1)

        optim.zero_grad()


        if use_boundary_2 and use_boundary_4 and use_boundary_8:
            out, out16, out32, detail2, detail4, detail8 = net(im)
        
        if (not use_boundary_2) and use_boundary_4 and use_boundary_8:
            out, out16, out32, detail4, detail8 = net(im)

        if (not use_boundary_2) and (not use_boundary_4) and use_boundary_8:
            out, out16, out32, detail8 = net(im)

        if (not use_boundary_2) and (not use_boundary_4) and (not use_boundary_8):
            out, out16, out32 = net(im)

 
        
        lossp = criteria_p(out, lb.long())
        loss2 = criteria_16(out16, lb.long())
        loss3 = criteria_32(out32, lb.long())
        
        boundery_bce_loss = 0.
        boundery_dice_loss = 0.
        
        
        if use_boundary_2: 
            # if dist.get_rank()==0:
            #     print('use_boundary_2')
            boundery_bce_loss2,  boundery_dice_loss2 = boundary_loss_func(detail2, lb)
            boundery_bce_loss += boundery_bce_loss2
            boundery_dice_loss += boundery_dice_loss2
        
        if use_boundary_4:
            # if dist.get_rank()==0:
            #     print('use_boundary_4')
            boundery_bce_loss4,  boundery_dice_loss4 = boundary_loss_func(detail4, lb)
            boundery_bce_loss += boundery_bce_loss4
            boundery_dice_loss += boundery_dice_loss4

        if use_boundary_8:
            # if dist.get_rank()==0:
            #     print('use_boundary_8')
            boundery_bce_loss8,  boundery_dice_loss8 = boundary_loss_func(detail8, lb)
            boundery_bce_loss += boundery_bce_loss8
            boundery_dice_loss += boundery_dice_loss8

        loss = lossp + loss2 + loss3 + boundery_bce_loss + boundery_dice_loss
        
        loss.backward()
        optim.step()

        loss_avg.append(loss.item())

        loss_boundery_bce.append(boundery_bce_loss.item())
        loss_boundery_dice.append(boundery_dice_loss.item())

        ## print training log message
        if (it+1)%msg_iter==0:
            loss_avg = sum(loss_avg) / len(loss_avg)
            lr = optim.lr
            ed = time.time()
            t_intv, glob_t_intv = ed - st, ed - glob_st
            eta = int((max_iter - it) * (glob_t_intv / it))
            eta = str(datetime.timedelta(seconds=eta))

            loss_boundery_bce_avg = sum(loss_boundery_bce) / len(loss_boundery_bce)
            loss_boundery_dice_avg = sum(loss_boundery_dice) / len(loss_boundery_dice)
            msg = ', '.join([
                'it: {it}/{max_it}',
                'lr: {lr:4f}',
                'loss: {loss:.4f}',
                'boundery_bce_loss: {boundery_bce_loss:.4f}',
                'boundery_dice_loss: {boundery_dice_loss:.4f}',
                'eta: {eta}',
                'time: {time:.4f}',
            ]).format(
                it = it+1,
                max_it = max_iter,
                lr = lr,
                loss = loss_avg,
                boundery_bce_loss = loss_boundery_bce_avg,
                boundery_dice_loss = loss_boundery_dice_avg,
                time = t_intv,
                eta = eta
            )
            
            logger.info(msg)
            loss_avg = []
            loss_boundery_bce = []
            loss_boundery_dice = []
            st = ed
            # print(boundary_loss_func.get_params())
        if (it+1)%save_iter_sep==0:# and it != 0:
            
            ## model
            logger.info('evaluating the model ...')
            logger.info('setup and restore model')
            
            net.eval()

            # ## evaluator
            logger.info('compute the mIOU')
            with torch.no_grad():
                single_scale1 = MscEvalV0()
                mIOU50 = single_scale1(net, dlval, n_classes)

                single_scale2= MscEvalV0(scale=0.75)
                mIOU75 = single_scale2(net, dlval, n_classes)


            save_pth = osp.join(save_pth_path, 'model_iter{}_mIOU50_{}_mIOU75_{}.pth'
            .format(it+1, str(round(mIOU50,4)), str(round(mIOU75,4))))
            
            state = net.module.state_dict() if hasattr(net, 'module') else net.state_dict()
#             if dist.get_rank()==0: 
            torch.save(state, save_pth)

            logger.info('training iteration {}, model saved to: {}'.format(it+1, save_pth))

            if mIOU50 > maxmIOU50:
                maxmIOU50 = mIOU50
                save_pth = osp.join(save_pth_path, 'model_maxmIOU50.pth'.format(it+1))
                state = net.module.state_dict() if hasattr(net, 'module') else net.state_dict()
#                 if dist.get_rank()==0: 
                torch.save(state, save_pth)
                    
                logger.info('max mIOU model saved to: {}'.format(save_pth))
            
            if mIOU75 > maxmIOU75:
                maxmIOU75 = mIOU75
                save_pth = osp.join(save_pth_path, 'model_maxmIOU75.pth'.format(it+1))
                state = net.module.state_dict() if hasattr(net, 'module') else net.state_dict()
#                 if dist.get_rank()==0: 
                torch.save(state, save_pth)
                logger.info('max mIOU model saved to: {}'.format(save_pth))
            
            logger.info('mIOU50 is: {}, mIOU75 is: {}'.format(mIOU50, mIOU75))
            logger.info('maxmIOU50 is: {}, maxmIOU75 is: {}.'.format(maxmIOU50, maxmIOU75))

            net.train()
    
    ## dump the final model
    save_pth = osp.join(save_pth_path, 'model_final.pth')
    net.cpu()
    state = net.module.state_dict() if hasattr(net, 'module') else net.state_dict()
#     if dist.get_rank()==0: 
    torch.save(state, save_pth)
    logger.info('training done, model saved to: {}'.format(save_pth))
    print('epoch: ', epoch)

In [ ]:
if __name__ == "__main__":
    train()